In [5]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu, evaluation_util as evu
import importlib
from typing import List, Dict, Any
import wrappers as wrp
import os
import numpy as np

In [3]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [6]:
scaling_data = pd.read_csv(os.path.join(EXT_DATA_DIR, 'train/scaling_factors.csv'), index_col=0).set_index('scene')

In [24]:
importlib.reload(wrp)
dataset = wrp.load_dataset(EXT_DATA_DIR)

In [25]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [26]:
default_sift_config = exu.SIFTConfig()
extractor_config = exu.FeatureExtractorConfig(default_sift_config)
extractor = exu.FeatureExtractor(config=extractor_config)

In [27]:
bf_config = exu.BFMatcherConfig(knn=True)
matcher_config = exu.MatcherConfig(bf_config)

In [28]:
list1 = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10), (11, 12), (13, 14), (15, 16)]
list2 = [(17, 18), (19, 20), (21, 22), (23, 24), (25, 26), (27, 28), (29, 30), (31, 32)]

In [29]:
importlib.reload(esu)

# Recreate the estimator configuration and estimator using the reloaded module
es_alg_conf = esu.MAGSACConfig()
es_conf = esu.EstimatorConfig('MAGSAC', es_alg_conf)
fe = esu.FundamentalMatrixEstimator(list1, list2, es_conf)


In [30]:
matcher = exu.FeatureMatcher(matcher_config)

In [31]:
dataset = wrp.preprocess_data(dataset, preprocessor, extractor)

In [32]:
wrp.sample_pairs_for_run(dataset, 1000)

[+] Processing scene "brandenburg_gate": found 6767 pairs (will keep 1000)
[+] Processing scene "trevi_fountain": found 11166 pairs (will keep 1000)
[+] Processing scene "lincoln_memorial_statue": found 6308 pairs (will keep 1000)
[+] Processing scene "pantheon_exterior": found 9888 pairs (will keep 1000)
[+] Processing scene "buckingham_palace": found 6646 pairs (will keep 1000)
[+] Processing scene "taj_mahal": found 6730 pairs (will keep 1000)
[+] Processing scene "temple_nara_japan": found 8115 pairs (will keep 1000)
[+] Processing scene "sagrada_familia": found 2767 pairs (will keep 1000)
[+] Processing scene "notre_dame_front_facade": found 8448 pairs (will keep 1000)
[+] Processing scene "colosseum_exterior": found 7314 pairs (will keep 1000)
[+] Processing scene "sacre_coeur": found 4233 pairs (will keep 1000)
[+] Processing scene "british_museum": found 889 pairs (will keep 889)


In [33]:
dataset = wrp.extract_features(dataset, extractor)

Scenes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:20<00:00,  6.74s/it]


In [34]:
wrp.match_features(dataset, matcher, covisibility_threshold=0.1)

Matching features for scene: brandenburg_gate
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene brandenburg_gate: 7.35 seconds
Matching features for scene: trevi_fountain
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene trevi_fountain: 58.83 seconds
Matching features for scene: lincoln_memorial_statue
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene lincoln_memorial_statue: 3.28 seconds
Matching features for scene: pantheon_exterior
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene pantheon_exterior: 28.71 seconds
Matching features for scene: buckingham_palace
In matcher there are 1000 valid pairs to estimate for
Total time taken for matching features in scene buckingham_palace: 51.67 seconds
Matching features for scene: taj_mahal
In matcher there are 1000 valid pairs to estimate

In [35]:
sub = wrp.estimate_fundamental_matrix(dataset=dataset, estimator=fe)

Estimating fundamental matrix for pairs in scene: brandenburg_gate
Estimating fundamental matrix for pairs in scene: trevi_fountain
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
Estimating fundamental matrix for pairs in scene: lincoln_memorial_statue
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, filling in random value!!
[-]Error: Unable to predict fundemental matrix, fillin

In [36]:
sub.head()

,sample_id,fundamental_matrix,mask,inliers1,inliers2
0,brandenburg_gate;95850075_1570642878-62661350_...,"[58.35728388845713, 102.47001496957583, 3.3900...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.141128, 0.006934], [-0.1361, 0.046888], [...","[[-0.057306, -0.201213], [-0.361891, 0.009507]..."
1,brandenburg_gate;95733753_243743733-54233019_1...,"[-24.59035673071758, -63.4177751688933, -12.19...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.286968, 0.038841], [-0.277438, -0.029953]...","[[-0.455103, 0.097561], [-0.435118, -0.05511],..."
2,brandenburg_gate;95048461_7775852580-62661350_...,"[-17.71485628738074, 7.777875050231567, -4.459...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.466916, -0.176258], [-0.447821, -0.235853...","[[-0.33407, -0.255708], [-0.200771, -0.266797]..."
3,brandenburg_gate;90920828_5082887495-74616849_...,"[-3.328306959868837, -39.13296128450056, 7.767...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.449611, -0.054365], [-0.44568, -0.184222]...","[[-0.456476, -0.087079], [-0.447788, -0.211336..."
4,brandenburg_gate;90920828_5082887495-27175827_...,"[-23.70566184595777, 103.95896073820806, 13.83...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ...","[[-0.44568, -0.184222], [-0.44463, -0.178125],...","[[-0.290195, -0.094119], [-0.289663, -0.090223..."


In [37]:
sub[sub.fundamental_matrix.isna()]

,sample_id,fundamental_matrix,mask,inliers1,inliers2


In [38]:
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

In [39]:
evals = wrp.evaluate_results(dataset, sub, scaling=scaling_data, thresholds_q=thresholds_q, thresholds_t=thresholds_t)

Evaluating for brandenburg_gate
Evaluating for trevi_fountain
Evaluating for lincoln_memorial_statue
Evaluating for pantheon_exterior
Evaluating for buckingham_palace
Evaluating for taj_mahal
Evaluating for temple_nara_japan
Evaluating for sagrada_familia
Evaluating for notre_dame_front_facade
Evaluating for colosseum_exterior
Evaluating for sacre_coeur
Evaluating for british_museum


In [42]:
# Extract scene names from sample_id
evals['scene'] = evals['sample_id'].apply(lambda x: x.split(';')[0])

# Group by scene and calculate mean accuracy measures
scene_accuracy = evals.groupby('scene').agg({
    'maa': 'mean'
}).reset_index()

scene_accuracy['maa'] = scene_accuracy['maa'].map(lambda x: f"{x:.2f}")
print(scene_accuracy)

                      scene   maa
0          brandenburg_gate  0.01
1            british_museum  0.02
2         buckingham_palace  0.00
3        colosseum_exterior  0.00
4   lincoln_memorial_statue  0.02
5   notre_dame_front_facade  0.01
6         pantheon_exterior  0.01
7               sacre_coeur  0.01
8           sagrada_familia  0.01
9                 taj_mahal  0.01
10        temple_nara_japan  0.02
11           trevi_fountain  0.01
